In [1]:
from feature_extractor import extract_string_lower
import os
from tqdm import tqdm
import pickle

In [2]:
from core.MV2 import MV2
from core.JIG import JIG
from core.SG2 import SG2

def GIPS(str_feature,
		 window_size, K, M, # MV2 파라미터
		 thetaJ,  # JIG 파라미터
		 vector_size, eps, minpts, ngram, hh1_size, hh2_size, hh3_size, ratio # SG2, AWL 파라미터
		):

	print(f'data no: {len(str_feature)}')

	str_feature = list(str_feature)
	for feature in str_feature:
		feature = list(feature)

	# 빅 그룹 식별
	minhashed_virtual_vectors = MV2(payloads=str_feature, window_size=window_size, K=K, M=M)

	big_group_indices = JIG(vectors=minhashed_virtual_vectors, thetaJ=thetaJ)
  
	
	big_group_payloads = []
	non_big_group_paylaods = []

	for idx, payload in enumerate(str_feature):
		if idx in big_group_indices:
			big_group_payloads.append(payload)
		else:
			non_big_group_paylaods.append(payload)
	
	# 시그니처 생성
	cluster_signatures = SG2(payloads=big_group_payloads, window_size=window_size, vector_size=vector_size, 
							 eps=eps, minpts=minpts, ngram=ngram, hh1_size=hh1_size, hh2_size=hh2_size, hh3_size=hh3_size, ratio=ratio)

	return cluster_signatures

In [52]:
malware_direc = './datasets/PE_malware/'
benign_direc = './datasets/ben/'

In [53]:
malware_path = os.listdir(malware_direc)

with open('./datasets/pkl/benign_md5.pkl', 'rb') as f:
    benign_path = pickle.load(f)


malware_path.remove('README.md')

print(len(malware_path), len(benign_path))

29909 29909


In [54]:
benign_strings = [] # 전체 스트링 저장
white = [] # train 화이트 리스트
benign_test = [] # test benign

for path_ in tqdm(benign_path[: int(len(benign_path) * 0.8)]):
	path = os.path.join(benign_direc, path_)
	
	res = extract_string_lower(path=path)
	benign_strings.append(res)
	white.append(res)

100%|██████████| 23927/23927 [07:43<00:00, 51.66it/s]


In [55]:
white_list = set()

for inner in tqdm(white):
    white_list.update(inner)
        
print(len(white_list))

100%|██████████| 23927/23927 [00:09<00:00, 2607.60it/s]

24033730


In [56]:
malware_strings = [] # 전체 스트링 저장
malware_train = [] # train 화이트 리스트
malware_test = [] # test malware
benign_test = [] # test benign

for path_ in tqdm(malware_path[: int(len(malware_path) * 0.8)]):
	path = os.path.join(malware_direc, path_)
	
	res = extract_string_lower(path=path)
	malware_strings.append(res)

	tmp = []
	for string in res:
		if string not in white_list:
			tmp.append(string)

	malware_train.append(tmp)

for path_ in tqdm(malware_path[int(len(malware_path) * 0.8): ]):
	path = os.path.join(malware_direc, path_)
	
	res = extract_string_lower(path=path)
	malware_strings.append(res)

	tmp = []
	for string in res:
		if string not in white_list:
			tmp.append(string)

	malware_test.append(tmp)

for path_ in tqdm(benign_path[int(len(benign_path) * 0.8): ]):
	path = os.path.join(benign_direc, path_)
	
	res = extract_string_lower(path=path)
	benign_strings.append(res)

	tmp = []
	for string in res:
		if string not in white_list:
			tmp.append(string)

	benign_test.append(tmp)

100%|██████████| 5982/5982 [02:08<00:00, 46.37it/s]


In [57]:
print(len(benign_strings), len(malware_strings))

with open('./datasets/pkl/low_benign_string.pkl', 'wb') as f:
    pickle.dump(benign_strings, f)
    
with open('./datasets/pkl/low_malware_string.pkl', 'wb') as f:
    pickle.dump(malware_strings, f)

29909 29909


In [3]:
# parameters

K = 64
M = 2 ** 14
thetaJ = 0.6
window_size = 4
vector_size = 512
eps = 0.4
minpts = 5
ngram = 4
hh1_size = 5000
hh2_size = 5000
hh3_size = 5000
ratio = 0.8

In [60]:
cluster_signatures = GIPS(str_feature=malware_train, window_size=window_size, K=K, M=M, thetaJ=thetaJ, vector_size=vector_size,
    eps=eps, minpts=minpts, ngram=ngram, hh1_size=hh1_size, hh2_size=hh2_size, hh3_size=hh3_size, ratio=ratio
)

data no: 23927
make minhashed vector


100%|██████████| 23927/23927 [02:45<00:00, 144.25it/s]


checking big group


100%|██████████| 23927/23927 [02:56<00:00, 135.94it/s]


chunking


100%|██████████| 15634/15634 [00:19<00:00, 803.10it/s]


start DBSCAN
end DBSCAN
make signature


100%|██████████| 18/18 [00:11<00:00,  1.59it/s]


end signature


In [61]:
maxV = 0

signatures = set()

for signature_list in cluster_signatures.values():
		for i in signature_list:
			signatures.add(i[0])
			maxV = max(maxV, len(i[0]))

print(maxV)

212982


In [62]:
print(len(signatures))

1661


In [63]:
for i in range(1, 20):
	res = []
	TP = 0
	FP = 0
	TN = 0
	FN = 0

	print('N', i)

	for strings in malware_test:
		cnt = 0
		for string in strings:
			if string in signatures:
				cnt += 1
		if cnt >= i:
			TP += 1
		else:
			FN += 1

	for strings in benign_test:
		cnt = 0
		for string in strings:
			if string in signatures:
				cnt += 1		
		if cnt < i:
			TN += 1
		else:
			TP += 1
		
	# print(sum(benign) / len(benign))
	print(f'TP: {TP}, FN: {FN}')
	print(f'FP: {FP}, TN: {TN}')

	presion = round(TP / (TP + FP) , 4)
	recall = round(TP / (TP + FN), 4)

	print(f'presion: {presion}, recall: {recall}')
	print('f1-score:', round(2 * (presion * recall) / (presion + recall), 4))
	print()

N 1
TP: 1791, FN: 4196
FP: 0, TN: 5977
presion: 1.0, recall: 0.2991
f1-score: 0.4605

N 2
TP: 1717, FN: 4265
FP: 0, TN: 5982
presion: 1.0, recall: 0.287
f1-score: 0.446

N 3
TP: 1717, FN: 4265
FP: 0, TN: 5982
presion: 1.0, recall: 0.287
f1-score: 0.446

N 4
TP: 1690, FN: 4292
FP: 0, TN: 5982
presion: 1.0, recall: 0.2825
f1-score: 0.4405

N 5
TP: 1642, FN: 4340
FP: 0, TN: 5982
presion: 1.0, recall: 0.2745
f1-score: 0.4308

N 6
TP: 1641, FN: 4341
FP: 0, TN: 5982
presion: 1.0, recall: 0.2743
f1-score: 0.4305

N 7
TP: 1640, FN: 4342
FP: 0, TN: 5982
presion: 1.0, recall: 0.2742
f1-score: 0.4304

N 8
TP: 1640, FN: 4342
FP: 0, TN: 5982
presion: 1.0, recall: 0.2742
f1-score: 0.4304

N 9
TP: 1616, FN: 4366
FP: 0, TN: 5982
presion: 1.0, recall: 0.2701
f1-score: 0.4253

N 10
TP: 1616, FN: 4366
FP: 0, TN: 5982
presion: 1.0, recall: 0.2701
f1-score: 0.4253

N 11
TP: 1616, FN: 4366
FP: 0, TN: 5982
presion: 1.0, recall: 0.2701
f1-score: 0.4253

N 12
TP: 1616, FN: 4366
FP: 0, TN: 5982
presion: 1.0, re

# THH 크기 늘리기

In [3]:
with open('./datasets/pkl/low_benign_string.pkl', 'rb') as f:
    benign_strings = pickle.load(f)
    
with open('./datasets/pkl/low_malware_string.pkl', 'rb') as f:
    malware_strings = pickle.load(f)

In [4]:
import random

random.Random(42).shuffle(benign_strings)
random.Random(42).shuffle(malware_strings)

In [5]:
white_list = dict()

for inner in tqdm(benign_strings[: int(len(benign_strings) * 0.8)]):
    for i in inner:
        if i not in white_list:
            white_list[i] = 0
        white_list[i] += 1
        
print(len(white_list))

100%|██████████| 23927/23927 [00:30<00:00, 792.42it/s] 

23910007


In [6]:
whitelist = set([i[0] for i in white_list.items() if i[1] > 0])
print(len(whitelist))

23910007


In [7]:
malware_train = [] # train 화이트 리스트
malware_test = [] # test malware
benign_test = []

for res in tqdm(malware_strings[: int(len(malware_strings) * 0.8)]):
	
	tmp = []
	for string in res:
		if string not in whitelist:
			tmp.append(string)

	malware_train.append(tmp)

for res in tqdm(malware_strings[int(len(malware_strings) * 0.8): ]):

	tmp = []
	for string in res:
		if string not in whitelist:
			tmp.append(string)

	malware_test.append(tmp)

for res in tqdm(benign_strings[int(len(benign_strings) * 0.8): ]):

	tmp = []
	for string in res:
		if string not in whitelist:
			tmp.append(string)

	benign_test.append(tmp)

100%|██████████| 5982/5982 [00:04<00:00, 1203.67it/s]


In [8]:
# parameters

K = 64
M = 2 ** 14
thetaJ = 0.6
window_size = 4
vector_size = 512
eps = 0.4
minpts = 5
ngram = 4
hh1_size = 15000
hh2_size = 15000
hh3_size = 5000
ratio = 0.4

In [9]:
cluster_signatures = GIPS(str_feature=malware_train, window_size=window_size, K=K, M=M, thetaJ=thetaJ, vector_size=vector_size,
    eps=eps, minpts=minpts, ngram=ngram, hh1_size=hh1_size, hh2_size=hh2_size, hh3_size=hh3_size, ratio=ratio
)

data no: 23927
make minhashed vector


100%|██████████| 23927/23927 [02:48<00:00, 141.85it/s]


checking big group


100%|██████████| 23927/23927 [02:13<00:00, 178.75it/s]


chunking


100%|██████████| 15659/15659 [00:19<00:00, 789.14it/s]


start DBSCAN
end DBSCAN
make signature


100%|██████████| 20/20 [00:20<00:00,  1.04s/it]


end signature


In [10]:
signatures = set()

for signature_list in cluster_signatures.values():
		print(len(signature_list))
		for i in signature_list:
			signatures.add(i[0])

print(len(signatures))

21
13025
71
19
40
33
28
3
16
10
16
145
33
13
18
1047
272
11
24
66
14899


In [12]:
for i in range(1, 20):
	res = []
	TP = 0
	FP = 0
	TN = 0
	FN = 0

	print('N', i)

	for strings in malware_test:
		cnt = 0
		for string in strings:
			if string in signatures:
				cnt += 1
		if cnt >= i:
			TP += 1
		else:
			FN += 1

	for strings in benign_test:
		cnt = 0
		for string in strings:
			if string in signatures:
				cnt += 1		
		if cnt >= i:
			FP += 1
		else:
			TN += 1
		
	# print(sum(benign) / len(benign))
	print(f'TP: {TP}\tFN: {FN}')
	print(f'FP: {FP}\t\tTN: {TN}')

	presion = round(TP / (TP + FP) , 4)
	recall = round(TP / (TP + FN), 4)

	print(f'presion: {presion}, recall: {recall}')
	print('f1-score:', round(2 * (presion * recall) / (presion + recall), 4))
	print()

N 1
TP: 4952	FN: 1030
FP: 64		TN: 5918
presion: 0.9872, recall: 0.8278
f1-score: 0.9005

N 2
TP: 4877	FN: 1105
FP: 16		TN: 5966
presion: 0.9967, recall: 0.8153
f1-score: 0.8969

N 3
TP: 4865	FN: 1117
FP: 6		TN: 5976
presion: 0.9988, recall: 0.8133
f1-score: 0.8966

N 4
TP: 4833	FN: 1149
FP: 2		TN: 5980
presion: 0.9996, recall: 0.8079
f1-score: 0.8936

N 5
TP: 4754	FN: 1228
FP: 2		TN: 5980
presion: 0.9996, recall: 0.7947
f1-score: 0.8855

N 6
TP: 4754	FN: 1228
FP: 2		TN: 5980
presion: 0.9996, recall: 0.7947
f1-score: 0.8855

N 7
TP: 4754	FN: 1228
FP: 2		TN: 5980
presion: 0.9996, recall: 0.7947
f1-score: 0.8855

N 8
TP: 4754	FN: 1228
FP: 2		TN: 5980
presion: 0.9996, recall: 0.7947
f1-score: 0.8855

N 9
TP: 4750	FN: 1232
FP: 2		TN: 5980
presion: 0.9996, recall: 0.794
f1-score: 0.885

N 10
TP: 4735	FN: 1247
FP: 2		TN: 5980
presion: 0.9996, recall: 0.7915
f1-score: 0.8835

N 11
TP: 4710	FN: 1272
FP: 2		TN: 5980
presion: 0.9996, recall: 0.7874
f1-score: 0.8809

N 12
TP: 4675	FN: 1307
FP: 2		

In [24]:
dark_list = dict()

for inner in tqdm(malware_strings[: int(len(malware_strings) * 0.8)]):
    for i in inner:
        if i not in dark_list:
            dark_list[i] = 0
        dark_list[i] += 1
        
print(len(dark_list))

100%|██████████| 23927/23927 [00:12<00:00, 1964.57it/s]

7684388


In [25]:
cnt = 0

dark_lists = []

for i in sorted([i for i in dark_list.items()], key=lambda x: -x[1]):
	if i[0] not in white_list:
		dark_lists.append(i)

print(len(dark_lists))

6694297


In [26]:
cnt = 0

cnt_200 = set()
for i in dark_lists:
    if i[1] > 250:
        cnt_200.add(i)

print(len(cnt_200))
print(len(dark_lists) - len(cnt_200))

for i in sorted([i for i in cnt_200], key=lambda x: -x[1]):
    print(i)

11111
6683186
("z-'6;2=", 1787)
('$vf(yu', 1787)
('w{|>d3', 1787)
('e|v:y)', 1787)
('}p]%uoc', 1787)
('g7<k[6', 1787)
('~6@wjv*', 1787)
(']zhd8ps', 1787)
('t7%0y|e', 1787)
('jb/)b3l', 1787)
('`a>|j>i', 1787)
('4m=t6d', 1787)
('{&8p^)j6', 1787)
('rkpe,i&', 1787)
("u|rv'u", 1787)
('2"1&a[', 1787)
(')ez>ah', 1787)
('9(5y\\u', 1787)
('yfv{y=t', 1787)
('xmcl+ok', 1787)
('"j81kt', 1787)
('a^0b\\^', 1787)
('{~y]tc', 1787)
('k,|niot', 1787)
('=ec&jl/p', 1787)
('ljt?y9>', 1787)
('yfwbw`', 1787)
('y~e`!&', 1787)
('o$yd*d', 1787)
('$;vxfj{>', 1787)
('dm+kx0c@[3', 1787)
('cjye}r', 1787)
('+kjh@s', 1787)
('l][tsc', 1787)
('tzgap)', 1787)
('dae!or', 1787)
('?>bw:,', 1787)
('shqat\\', 1787)
('mn6}pi', 1787)
('~u)owo', 1787)
('-h/kwoa', 1787)
('&eh5{_', 1787)
('|/<$yae', 1787)
('lsajre', 1787)
('1d!3jr', 1787)
('s|{)d(', 1787)
('ouktuf', 1787)
('@tsv.e', 1787)
('jk/|l5a!*', 1787)
('|^ow>m_', 1787)
('c[m@xv', 1787)
('axaiwl', 1787)
('ycjo_^', 1787)
('$fkx$(', 1787)
('l:kdgm`', 1787)
('{l[x2xh', 1787)
(

In [27]:
cnt_200s = set([i[0] for i in cnt_200])

cnt = 0

for inner in tqdm(malware_strings[: int(len(malware_strings) * 0.8)]):
	for i in inner:
		if i in cnt_200s:
			cnt += 1
			break

print(cnt)

100%|██████████| 23927/23927 [00:00<00:00, 36188.96it/s]

20963
